# Image classification with small data

1. Download and setup dataset (Kaggle Cats & Dogs) https://www.kaggle.com/c/dogs-vs-cats/data
2. Train a small convnet on our small data (acc ~ 0.75)
3. Train a MLP using bottleneck features of pretrained model (acc ~ 0.90)
4. Finetuning top layers of pretrained model on our small data (acc ~ 0.94)


In [3]:
# imports
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import applications
import numpy as np
from keras import Model
from keras import optimizers
import matplotlib.pyplot as plt

/home/mathlizard/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
# define plot & parse functions
def parse_log_file(pfile):
    f= open(pfile).readlines()
    lines = [i.strip() for i in f]
    log= {k:[] for k in ['acc','val_acc','loss','val_loss']}
    for l in lines:
        if 'loss' in l:
            for k in log.keys():
                log[k].append(float(l.split(k+': ')[1].split(' -')[0]))
    return log 

def plot_loss_acc(pfile):
    history = parse_log_file(pfile)
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    plt.title('model acc')
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [8]:
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)

## Train a small convnet on our small data

In [4]:
# define a sequential model (small conv net): 
# 3 conv blocks (Conv2D, Activation('relu'), MaxPooling2D) + 2 dense layers
# Conv_1: filters 32, kernel size(3,3)
# Conv_2: filters 32, kernel size(3,3)
# Conv_3: filters 64, kernel size(3,3)
# Flatten
# Dense_1: 64
# Activation('relu')
# Dropout(0.5)
# Dense_2:  ? 
# Activation('sigmoid')

# print model summary

# # add your implementation

model = Sequential()

model.add(Conv2D(
32, (3,3), 
activation = 'relu',
input_shape=(150,150,3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(
32, (3,3), 
activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(
32, (3,3), 
activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9248)              0         
__________

In [6]:
# compile model 
# use binary crossentropy loss 
# and rmsprop optimizer

# # add your implementation

model.compile(optimizer=optimizers.rmsprop(),
             loss = 'binary_crossentropy',
             metrics=['accuracy'])


In [9]:
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [10]:
# define a keras ImageDataGenerator for training data with appropriate augmentation 
# use rescale=1. / 255 to normalise pixles values

# define a keras ImageDataGenerator for test data  (no augmentation only rescaling)

# # add your implementation
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)


In [11]:
# define training and validation iterators 
# use ImageDataGenerator.flow_from_directory to training and validation dirs accordingly 

# # add your implementation
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode="binary"
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [12]:
# train and validate the model using fit_generator
history = model.fit_generator(
train_generator,
steps_per_epoch=100,
epochs=10,
validation_data=validation_generator,
validation_steps=50)

# # add your implementation

Epoch 1/10
100/100 [==============================] - 15s 148ms/step - loss: 0.6978 - acc: 0.5455 - val_loss: 0.6743 - val_acc: 0.6325
Epoch 2/10
100/100 [==============================] - 14s 143ms/step - loss: 0.6462 - acc: 0.6355 - val_loss: 0.6192 - val_acc: 0.6575
Epoch 3/10
100/100 [==============================] - 15s 152ms/step - loss: 0.5956 - acc: 0.6850 - val_loss: 0.5751 - val_acc: 0.6912
Epoch 4/10
100/100 [==============================] - 15s 149ms/step - loss: 0.5538 - acc: 0.7270 - val_loss: 0.5693 - val_acc: 0.7037
Epoch 5/10
100/100 [==============================] - 15s 151ms/step - loss: 0.5175 - acc: 0.7630 - val_loss: 0.5777 - val_acc: 0.6862
Epoch 6/10
100/100 [==============================] - 15s 154ms/step - loss: 0.4617 - acc: 0.7840 - val_loss: 0.6068 - val_acc: 0.6988
Epoch 7/10
100/100 [==============================] - 15s 155ms/step - loss: 0.4259 - acc: 0.8000 - val_loss: 0.5474 - val_acc: 0.7238
Epoch 8/10
100/100 [==============================] - 1

## Train a MLP using bottleneck features
### MLP -- multilayer percepton

In [14]:
# define VGG16 network using keras applications.VGG16  
# set weights = 'imagenet'
# set include_top=False : don't include the fully-connected layer at the top of the network 
# set input_shape: image shape

# # add your implementation
from keras.applications import VGG16

vgg_16=VGG16(include_top=False, input_shape=(150,150,3), weights='imagenet')
# define a keras ImageDataGenerator for data (no augmentation only rescaling)
# define training and validation iterators same as before (set shuffle=False)

# # add your implementation

train_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode="binary",
    shuffle=False
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary',
    shuffle=False
)
# extract image features for training and validation separately using predict_generator

# # add your implementation
pred_train = vgg_16.predict_generator(train_generator)
pred_valid= vgg_16.predict_generator(validation_generator)





58892288/58889256 [==============================] - 22s 0us/step
Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [17]:
# save training and validation features

# # add your implementation

np.save(open('bottleneck_features_train_1.npy', 'wb'), pred_train)
np.save(open('bottleneck_features_validation_1.npy', 'wb'), pred_valid)
pred_train.shape


(2000, 4, 4, 512)

In [ ]:
np.save(open('training_features.npy', 'wb'),)

In [19]:
train_generator.classes

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [26]:
# create training and validation labels array (features are in order because we used shuffle=false) 

# # add your implementation
train_labels = train_generator.classes
validation_labels = validation_generator.classes

# define model of two dense layers 256 and ? 
# model should start with Flatten layer (to flatten extracted image features to a vector)
# don’t forget activations and dropout(0.5)
# compile with binary_crossentropy loss and rmsprop optimizer

# # add your implementation
model=Sequential()
model.add(Flatten(input_shape=(4,4,512)))

model.add(Dense(256, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='rmsprop',
             loss = 'binary_crossentropy',
             metrics=['accuracy'])
# Train MLP using fit function

# # add your implementation


history = model.fit(
pred_train,
    train_labels,
epochs=10,
validation_data=(pred_valid, validation_labels))
# save weights to use them in fine-tuning later on

# # add your implementation

top_model_weights_path = 'bottleneck_fc_path'


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)               2097408   
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 257       
Total params: 2,097,665
Trainable params: 2,097,665
Non-trainable params: 0
_________________________________________________________________
Train on 2000 samples, validate on 800 samples
Epoch 1/10
2000/2000 [==============================] - 2s 1ms/step - loss: 0.8516 - acc: 0.7275 - val_loss: 0.4069 - val_acc: 0.7913
Epoch 2/10
2000/2000 [==============================] - 1s 739us/step - loss: 0.374

In [30]:

vgg_16.input
model.output

<tf.Tensor 'dense_16/Sigmoid:0' shape=(?, 1) dtype=float32>

## Finetuning top layers of pretrained model on our small data

In [34]:
# define new model:  VGG16 as base and MLP as top 
# input = base_model.input, output=top_model(base_model.output)

# # add your implementation


model = Model(inputs= [vgg_16.input], outputs=vgg16[model.output])
# vgg_16.get_layer(index=)


# freeze the first 15 layers (up to the last conv block)
# set trainable=false (weights will not be updated)

# # add your implementation

# compile with binary_crossentropy loss and SGD with low learing rate optimizer

# # add your implementation

ValueError: Graph disconnected: cannot obtain value for tensor Tensor("flatten_8_input:0", shape=(?, 4, 4, 512), dtype=float32) at layer "flatten_8_input". The following previous layers were accessed without issue: []

In [ ]:
# fine-tune the model using fit_generator and train and validation iterators